# Module 3: Building DevAssist - Full Dev Workflow

## Multi-Agent Development System

In Modules 1 and 2, we built the Project Analyzer and Feature Implementer. Now we'll combine them into a **complete multi-agent development workflow** with specialized subagents.

### What We've Built

| Module | Component | Capability |
|--------|-----------|------------|
| Module 1 | Project Analyzer | Understand any codebase |
| Module 2 | Feature Implementer | Make code changes with tracking |
| **Module 3** | **Full Dev Workflow** | **Multi-agent: research, implement, test, document** |

### This Module's Goals

1. Build specialized subagents (Researcher, Implementer, Tester, DocWriter)
2. Create a coordinator that orchestrates them
3. Implement context isolation for security
4. Run subagents in parallel for performance
5. Build a complete feature with the full pipeline

### The Architecture

```
DevAssist Coordinator
         |
         +---> Researcher    (Read, Glob, Grep)
         |         Find relevant code and patterns
         |
         +---> Implementer   (Read, Edit, TodoWrite)
         |         Make code changes
         |
         +---> Tester        (Bash, Read)
         |         Run tests, validate
         |
         +---> DocWriter     (Read, Edit)
                   Update documentation
```

---

## 1. Setup

In [ ]:
import os
import asyncio
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your Anthropic API key: ")

In [ ]:
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    AssistantMessage,
    ToolUseMessage,
    ToolResultMessage,
    ResultMessage
)

PROJECT_PATH = os.path.abspath("../sample_project")
print(f"Working with TaskFlow at: {PROJECT_PATH}")

---

## 2. Exercise 1: Defining Specialized Subagents

Each subagent has a specific role with appropriate tools and permissions.

### The Principle: Least Privilege

> Give each subagent only the tools it needs - nothing more.

In [ ]:
# Define our specialized subagents
SUBAGENTS = {
    "researcher": {
        "description": "Gathers context and finds relevant code",
        "tools": ["Read", "Glob", "Grep"],
        "permission_mode": "bypassPermissions",  # Safe: read-only
        "system_prompt": """You are a Research Specialist for DevAssist.

Your role:
- Find relevant files and code patterns
- Analyze project structure and dependencies
- Identify existing patterns to follow
- Report findings clearly and concisely

You can ONLY read - never suggest making changes.
Focus on gathering information that helps others do their jobs."""
    },
    
    "implementer": {
        "description": "Makes code changes following patterns",
        "tools": ["Read", "Glob", "Edit", "TodoWrite"],
        "permission_mode": "acceptEdits",  # Auto-approve edits
        "system_prompt": """You are an Implementation Specialist for DevAssist.

Your role:
- Implement features based on research findings
- Follow existing code patterns exactly
- Use TodoWrite to track your progress
- Make minimal, focused changes

Rules:
- Match the existing code style
- Add docstrings to new functions
- Don't change unrelated code"""
    },
    
    "tester": {
        "description": "Runs tests and validates changes",
        "tools": ["Bash", "Read", "Grep"],
        "permission_mode": "bypassPermissions",
        "system_prompt": """You are a Testing Specialist for DevAssist.

Your role:
- Run existing tests with pytest
- Check for syntax errors
- Verify new code doesn't break anything
- Report test results clearly

You can run tests but NOT modify any files."""
    },
    
    "docwriter": {
        "description": "Updates documentation",
        "tools": ["Read", "Glob", "Edit"],
        "permission_mode": "acceptEdits",
        "system_prompt": """You are a Documentation Specialist for DevAssist.

Your role:
- Update README when features are added
- Add usage examples
- Keep documentation concise and clear

Only update documentation files (README.md, etc.)
Do NOT modify code files."""
    }
}

# Display the configuration
print("DevAssist Subagent Configuration")
print("=" * 50)
for name, config in SUBAGENTS.items():
    print(f"\n{name.upper()}")
    print(f"  Role: {config['description']}")
    print(f"  Tools: {', '.join(config['tools'])}")
    print(f"  Permission: {config['permission_mode']}")

### Context Isolation Matrix

| Subagent | Read | Edit | Bash | Why |
|----------|------|------|------|-----|
| Researcher | Yes | No | No | Only gathers info |
| Implementer | Yes | Yes | No | Makes code changes |
| Tester | Yes | No | Yes | Runs tests |
| DocWriter | Yes | Yes | No | Updates docs |

---

## 3. Exercise 2: Running Individual Subagents

Let's create a function to run any subagent and see them in action.

In [ ]:
async def run_subagent(name: str, task: str, project_path: str, verbose: bool = True):
    """Run a specialized subagent."""
    
    if name not in SUBAGENTS:
        raise ValueError(f"Unknown subagent: {name}")
    
    config = SUBAGENTS[name]
    
    options = ClaudeAgentOptions(
        allowed_tools=config["tools"],
        permission_mode=config["permission_mode"],
        system_prompt=config["system_prompt"]
    )
    
    full_prompt = f"{task}\n\nProject path: {project_path}"
    
    result = ""
    tools_used = []
    
    if verbose:
        print(f"[{name.upper()}] Starting...")
    
    async for msg in query(prompt=full_prompt, options=options):
        if isinstance(msg, ToolUseMessage):
            tools_used.append(msg.tool_name)
            if verbose:
                print(f"  [{name}] Using {msg.tool_name}...")
        elif isinstance(msg, AssistantMessage) and msg.content:
            result = msg.content
    
    if verbose:
        print(f"[{name.upper()}] Complete ({len(tools_used)} tool calls)")
    
    return {
        "agent": name,
        "result": result,
        "tools_used": tools_used
    }

In [ ]:
# Test the researcher
research = await run_subagent(
    "researcher",
    "Find all the CLI commands available in TaskFlow and how they're structured",
    PROJECT_PATH
)

print("\nResearch Result:")
print(research["result"][:800])

In [ ]:
# Test the tester
test_result = await run_subagent(
    "tester",
    "Run the existing tests and report the results",
    PROJECT_PATH
)

print("\nTest Result:")
print(test_result["result"][:600])

---

## 4. Exercise 3: The DevAssist Coordinator

The coordinator orchestrates all subagents to complete a feature end-to-end.

In [ ]:
class DevAssistCoordinator:
    """Orchestrates subagents to implement features end-to-end."""
    
    def __init__(self, project_path: str):
        self.project_path = project_path
        self.results = {}
    
    async def implement_feature(self, feature_description: str):
        """Full pipeline: research -> implement -> test -> document."""
        
        print("="*70)
        print(f"DEVASSIST: Implementing Feature")
        print(f"Feature: {feature_description}")
        print("="*70 + "\n")
        
        # Phase 1: Research
        print("PHASE 1: RESEARCH")
        print("-" * 40)
        self.results["research"] = await run_subagent(
            "researcher",
            f"""Research the codebase to prepare for implementing this feature:
            {feature_description}
            
            Find:
            1. Related existing code patterns
            2. Files that need to be modified
            3. How similar features are implemented""",
            self.project_path
        )
        print(f"\nFindings summary: {self.results['research']['result'][:300]}...\n")
        
        # Phase 2: Implementation
        print("PHASE 2: IMPLEMENTATION")
        print("-" * 40)
        self.results["implementation"] = await run_subagent(
            "implementer",
            f"""Implement this feature:
            {feature_description}
            
            Context from research:
            {self.results['research']['result'][:500]}
            
            Follow the existing patterns exactly.""",
            self.project_path
        )
        print(f"\nImplementation summary: {self.results['implementation']['result'][:300]}...\n")
        
        # Phase 3: Testing
        print("PHASE 3: TESTING")
        print("-" * 40)
        self.results["testing"] = await run_subagent(
            "tester",
            """Run all tests and check for any issues:
            1. Run pytest on the tests directory
            2. Check if there are any syntax errors
            3. Report the results""",
            self.project_path
        )
        print(f"\nTest results: {self.results['testing']['result'][:300]}...\n")
        
        # Phase 4: Documentation
        print("PHASE 4: DOCUMENTATION")
        print("-" * 40)
        self.results["documentation"] = await run_subagent(
            "docwriter",
            f"""Update the README.md to document this new feature:
            {feature_description}
            
            Add a usage example in the appropriate section.""",
            self.project_path
        )
        print(f"\nDoc update: {self.results['documentation']['result'][:300]}...\n")
        
        # Summary
        print("="*70)
        print("DEVASSIST: FEATURE COMPLETE")
        print("="*70)
        total_tools = sum(len(r["tools_used"]) for r in self.results.values())
        print(f"Total tool calls: {total_tools}")
        print(f"Phases completed: {len(self.results)}")
        
        return self.results

In [ ]:
# Run the full pipeline to add a new feature
coordinator = DevAssistCoordinator(PROJECT_PATH)

results = await coordinator.implement_feature(
    """Add an 'export' command that exports all tasks to a JSON file.
    Usage: python task_manager.py export tasks_backup.json"""
)

---

## 5. Exercise 4: Parallel Subagent Execution

When tasks are independent, run them in parallel for better performance.

In [ ]:
async def parallel_analysis(project_path: str):
    """Run multiple analysis tasks in parallel."""
    
    print("Running parallel analysis...\n")
    
    # Define independent analysis tasks
    tasks = [
        run_subagent("researcher", "List all Python files and their purposes", project_path, verbose=False),
        run_subagent("researcher", "Find all TODO comments in the codebase", project_path, verbose=False),
        run_subagent("tester", "Run tests and report results", project_path, verbose=False),
    ]
    
    # Run all in parallel
    import time
    start = time.time()
    results = await asyncio.gather(*tasks)
    elapsed = time.time() - start
    
    print(f"Completed 3 tasks in {elapsed:.2f}s (parallel)\n")
    
    # Display results
    labels = ["File Analysis", "TODO Search", "Test Results"]
    for label, result in zip(labels, results):
        print(f"=== {label} ===")
        print(result["result"][:400])
        print()
    
    return results

In [ ]:
# Run parallel analysis
parallel_results = await parallel_analysis(PROJECT_PATH)

### When to Use Parallel vs Sequential

| Scenario | Approach | Why |
|----------|----------|-----|
| Independent analysis tasks | Parallel | No dependencies |
| Research then implement | Sequential | Implementation needs research |
| Multiple file reads | Parallel | Independent operations |
| Edit then test | Sequential | Test needs completed edits |

---

## 6. Exercise 5: Error-Tolerant Pipeline

Production systems need to handle failures gracefully.

In [ ]:
async def safe_run_subagent(name: str, task: str, project_path: str):
    """Run subagent with error handling."""
    try:
        return await run_subagent(name, task, project_path, verbose=False)
    except Exception as e:
        print(f"  [ERROR] {name} failed: {e}")
        return {
            "agent": name,
            "result": f"Error: {str(e)}",
            "tools_used": [],
            "success": False
        }


async def resilient_pipeline(project_path: str, feature: str):
    """Pipeline that continues even if some steps fail."""
    
    print(f"Running resilient pipeline for: {feature}\n")
    
    steps = [
        ("researcher", f"Research how to: {feature}"),
        ("implementer", f"Implement: {feature}"),
        ("tester", "Run tests"),
        ("docwriter", f"Document: {feature}")
    ]
    
    results = []
    
    for agent, task in steps:
        print(f"Running {agent}...")
        result = await safe_run_subagent(agent, task, project_path)
        result["success"] = not result["result"].startswith("Error:")
        results.append(result)
    
    # Summary
    successful = sum(1 for r in results if r.get("success", True))
    print(f"\nPipeline complete: {successful}/{len(results)} steps succeeded")
    
    return results

In [ ]:
# Test resilient pipeline
# results = await resilient_pipeline(PROJECT_PATH, "Add a clear command to delete all completed tasks")

---

## 7. Exercise 6: Complete DevAssist System

Let's put it all together into a complete system.

In [ ]:
class DevAssist:
    """Complete AI Development Assistant."""
    
    def __init__(self, project_path: str):
        self.project_path = project_path
        self.history = []
    
    async def analyze(self):
        """Analyze the project (Module 1 capability)."""
        print("DevAssist: Analyzing project...\n")
        result = await run_subagent(
            "researcher",
            """Provide a complete analysis of this project:
            1. What does it do?
            2. What's the tech stack?
            3. What are the main components?
            4. What features does it have?""",
            self.project_path
        )
        self.history.append({"action": "analyze", "result": result})
        return result["result"]
    
    async def implement(self, feature: str):
        """Implement a feature (Module 2 capability)."""
        print(f"DevAssist: Implementing '{feature}'...\n")
        result = await run_subagent(
            "implementer",
            f"Implement this feature following existing patterns: {feature}",
            self.project_path
        )
        self.history.append({"action": "implement", "feature": feature, "result": result})
        return result["result"]
    
    async def test(self):
        """Run tests."""
        print("DevAssist: Running tests...\n")
        result = await run_subagent(
            "tester",
            "Run all tests and report results",
            self.project_path
        )
        self.history.append({"action": "test", "result": result})
        return result["result"]
    
    async def full_feature(self, feature: str):
        """Complete feature implementation with all phases (Module 3 capability)."""
        coordinator = DevAssistCoordinator(self.project_path)
        results = await coordinator.implement_feature(feature)
        self.history.append({"action": "full_feature", "feature": feature, "results": results})
        return results
    
    def status(self):
        """Show session status."""
        print(f"DevAssist Session Status")
        print(f"Project: {self.project_path}")
        print(f"Actions performed: {len(self.history)}")
        for i, item in enumerate(self.history, 1):
            print(f"  {i}. {item['action']}")

In [ ]:
# Create DevAssist instance
devassist = DevAssist(PROJECT_PATH)

# Analyze the project
analysis = await devassist.analyze()
print("\nAnalysis:")
print(analysis[:600])

In [ ]:
# Check status
devassist.status()

---

## 8. MCP Server Integration (Bonus)

For extending beyond built-in tools, you can create MCP servers.

In [ ]:
# MCP server configuration example
mcp_example = '''
# To add custom tools via MCP:

options = ClaudeAgentOptions(
    allowed_tools=["Read", "mcp__myserver__custom_tool"],
    mcp_servers={
        "myserver": {
            "command": "python",
            "args": ["path/to/server.py"]
        }
    }
)

# Tool naming: mcp__<server-name>__<tool-name>
# Example: mcp__github__create_issue
'''

print("MCP Integration Example:")
print(mcp_example)

---

## Summary

In this module, you built the complete **DevAssist** system with multi-agent architecture.

### What You Learned

| Concept | Implementation |
|---------|----------------|
| Subagent Definition | Specialized configs with isolated tools |
| Context Isolation | Least privilege per agent |
| Coordinator Pattern | Orchestrate multiple subagents |
| Parallel Execution | `asyncio.gather()` for independent tasks |
| Error Handling | Resilient pipelines that continue on failure |

### The Complete DevAssist System

```
DevAssist
   |
   +-- analyze()      -> Researcher
   +-- implement()    -> Implementer  
   +-- test()         -> Tester
   +-- full_feature() -> All 4 agents in sequence
```

### Subagent Isolation

| Agent | Tools | Why |
|-------|-------|-----|
| Researcher | Read, Glob, Grep | Only reads |
| Implementer | Read, Edit, TodoWrite | Makes code changes |
| Tester | Bash, Read | Runs commands |
| DocWriter | Read, Edit | Updates docs only |

### Production Patterns

```python
# Sequential pipeline (dependent tasks)
research = await run_subagent("researcher", task, path)
impl = await run_subagent("implementer", task, path)

# Parallel execution (independent tasks)
results = await asyncio.gather(
    run_subagent("researcher", task1, path),
    run_subagent("researcher", task2, path)
)

# Error-tolerant
try:
    result = await run_subagent(...)
except Exception as e:
    result = {"error": str(e)}
```

### You've Built

A complete AI development assistant that can:
1. **Analyze** any codebase
2. **Plan** feature implementations
3. **Implement** code changes
4. **Test** and validate
5. **Document** changes

All with proper isolation, parallel execution, and error handling!